title and description

In [ ]:
import os
#os.environ['create_image']='True'
#os.environ['repository']='romeokienzler'
os.environ['install_requirements']='True'


In [ ]:
import os
if bool(os.environ.get('create_image',False)):
    docker_file="""
    FROM registry.access.redhat.com/ubi8/python-38
    RUN pip install nvflare==2.0.16
    RUN pip install tensorflow==2.9.1
    ADD nvflare.ipynb /
    ENTRYPOINT ["ipython","/nvflare.ipynb"]
    """
    with open("Dockerfile", "w") as text_file: 
        text_file.write(docker_file)

    !docker build -t claimed-train-nvflare .
    !docker tag claimed-train-nvflare `echo $repository`/claimed-train-nvflare
    !docker push `echo $repository`/claimed-train-nvflare
elif bool(os.environ.get('install_requirements',False)):
    !pip install tensorflow==2.9.1 tensorflow-hub==0.12.0

In [ ]:
import os
import tensorflow as tf
# from tensorflow.keras.applications import ResNet50V2 # , MobileNetV3Small
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras import Model
from claimed_utils import unzip, zipdir
import os.path
import glob
from io import BytesIO
from minio import Minio
from ansible.module_utils.parsing.convert_bool import boolean

In [ ]:
# @dependency codait_utils.ipynb
# @param image_shape shape the images shall be scaled to and the
# model then will accept
# @param model zip file name
# @param data zip file name
# @param model folder name
# @param data folder name
# @param epochs number of epochs to train
# @param checkpoint activate checkpointing
# @param checkpoint_ip minio endpoint
# @param checkpoint_user minio user
# @param checkpoint_pass minio pw
# @param checkpoint_bucket minio bucket
# @returns model zip file

In [ ]:
image_shape = os.environ.get('image_shape', '400,400')
model_zip = os.environ.get('model_zip', 'model.zip')
data_zip = os.environ.get('data_zip', 'data.zip')
model_folder = os.environ.get('model', 'model')
data = os.environ.get('data', 'data')
epochs = int(os.environ.get('epochs', 1))
checkpoint = boolean(os.environ.get('checkpoint', False))
checkpoint_ip = os.environ.get('checkpoint_ip')
checkpoint_user = os.environ.get('checkpoint_user', 'minio')
checkpoint_pass = os.environ.get('checkpoint_pass', 'minio123')
checkpoint_bucket = os.environ.get('checkpoint_bucket', 'checkpoint')

In [ ]:
exists = False

if checkpoint:
    client = Minio(checkpoint_ip,
                   checkpoint_user,
                   checkpoint_pass,
                   secure=False)

    objects = client.list_objects(checkpoint_bucket)
    asset_name = model_zip
    for obj in objects:
        if asset_name == obj.object_name:
            exists = True
            client.fget_object(checkpoint_bucket, model_zip, model_zip)
            break

In [ ]:
import numpy as np
# TF1 version
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
tf.config.run_functions_eagerly(False)


In [ ]:
model_handle = 'https://tfhub.dev/google/tiny_video_net/tvn2/1'
model = hub.Module(model_handle)


In [ ]:

vid_placeholder = tf.placeholder(tf.float32,
                                 shape=(batch_size * num_frames,
                                        image_size, image_size, 3))



In [ ]:
# This generates a random video. It should be replaced by a user's video.
# video = load_video(video_path) which should return a video of the above shape.
video = np.random.rand(*vid_placeholder.shape)



In [ ]:
predictions = model(video)


In [ ]:
model_link='https://tfhub.dev/google/tiny_video_net/tvn2/1'
model_link=$model_link'?tf-hub-format=compressed'
wget $model_link -O model
tar xvzf model
rm model

In [ ]:
%%bash
mkdir tvn2
mv assets tvn2/
mv variables tvn2/
mv *.pb tvn2/

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
tf.compat.v1.enable_resource_variables()
model = hub.load("tvn2")


In [ ]:

vid_placeholder = tf.placeholder(tf.float32,
                                 shape=(batch_size * num_frames,
                                        image_size, image_size, 3))
